Southern University of Science and Technology-Department of Computer Science and Engineering

Course: Machine Learning(CS 405)-Professor: Qi Hao

## Homework #2
#### Due date: October, 7th, 2019

In [ ]:
"""
Import libraries that you might require.

"""

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import operator

from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings('ignore')

We will implement the KNN algorithm for the breast cancer dataset. Refer to the pdf and the following functions for the instructions. Complete all the functions as indicated below. The four functions would be autograded as mentioned in the pdf.

In [ ]:
"""
Task 1: Classification

Please implement KNN for K: 3, 5, and 7 with the following norms:
L1
L2
L-inf
"""

from itertools import product
from typing import Counter
from sklearn.datasets import load_breast_cancer

breast = load_breast_cancer()

X = breast['data']
y = breast['target']

np.random.seed(100)
p = np.random.permutation(len(X))
X, y = X[p], y[p]

X_train, y_train = X[:400], y[:400]
X_val, y_val = X[400:500], y[400:500]
X_test, y_test = X[500:], y[500:]


def distanceFunc(metric_type, vec1, vec2):
    """
    Computes the distance between two d-dimension vectors. 
    
    Please DO NOT use Numpy's norm function when implementing this function. 
    
    Args:
        metric_type (str): Metric: L1, L2, or L-inf
        vec1 ((d,) np.ndarray): d-dim vector
        vec2 ((d,)) np.ndarray): d-dim vector
    
    Returns:
        distance (float): distance between the two vectors
    """

    if metric_type == "L1":
        distance = np.sum(np.abs(vec1 - vec2))

    if metric_type == "L2":
        distance = np.sqrt(np.sum(np.square(vec1 - vec2)))

    if metric_type == "L-inf":
        distance = np.max(np.abs(vec1 - vec2))

    return distance


def computeDistancesNeighbors(K, metric_type, X_train, y_train, sample):
    """
    Compute the distances between every datapoint in the train_data and the 
    given sample. Then, find the k-nearest neighbors.
    
    Return a numpy array of the label of the k-nearest neighbors.
    
    Args:
        K (int): K-value
        metric_type (str): metric type
        X_train ((n,p) np.ndarray): Training data with n samples and p features
        y_train : Training labels
        sample ((p,) np.ndarray): Single sample whose distance is to computed with every entry in the dataset
        
    Returns:
        neighbors (list): K-nearest neighbors' labels
    """

    dist = [distanceFunc(metric_type, sample, X_train[i]) for i in range(len(X_train))]
    return [y_train[idx] for idx in np.argsort(dist, kind='stable')[:K]]


def Majority(neighbors):
    """
    Performs majority voting and returns the predicted value for the test sample.
    
    Since we're performing binary classification the possible values are [0,1].
    
    Args:
        neighbors (list): K-nearest neighbors' labels
        
    Returns:
        predicted_value (int): predicted label for the given sample
    """
    cnt = Counter(neighbors)
    return cnt.most_common(1)[0][0]  # fixme


def KNN(K, metric_type, X_train, y_train, X_val):
    """
    Returns the predicted values for the entire validation or test set.
    
    Please DO NOT use Scikit's KNN model when implementing this function. 

    Args:
        K (int): K-value
        metric_type (str): metric type
        X_train ((n,p) np.ndarray): Training data with n samples and p features
        y_train : Training labels
        X_val ((n, p) np.ndarray): Validation or test data
        
    Returns:
        predicted_values (list): output for every entry in validation/test dataset 
    """
    
    return [Majority(computeDistancesNeighbors(K, metric_type, X_train, y_train, sample)) for sample in X_val]


def evaluation(predicted_values, actual_values):
    """
    Computes the accuracy of the given datapoints.
    
    Args:
        predicted_values ((n,) np.ndarray): Predicted values for n samples
        actual_values ((n,) np.ndarray): Actual values for n samples
    
    Returns:
        accuracy (float): accuracy
    """
    
    return accuracy_score(predicted_values, actual_values)


df = pd.DataFrame(columns=['K', 'norm', 'val-accuracy', 'test-accuracy'])

def main():
    global df
    """
    Calls the above functions in order to implement the KNN algorithm.
    
    Test over the following range K = 3,5,7 and all three metrics. 
    In total you will have nine combinations to try.
    
    PRINTS out the accuracies for the nine combinations on the validation set,
    and the accuracy on the test set for the selected K value and appropriate norm.
    
    REMEMBER: You have to report these values by populating the Table 1.
    """

    K = [3,5,7]
    norm = ["L1", "L2", "L-inf"]
    
    print("<<<<VALIDATION DATA PREDICTIONS>>>>")
    for k, n in product(K, norm):
        acc = evaluation(KNN(k, n, X_train, y_train, X_val), y_val)
        print(f'K = {k}, Norm = {n}, Accuracy = {acc}')
        df = df.append({'K': k, 'norm': n, 'val-accuracy': acc}, ignore_index=True)

    print("<<<<TEST DATA PREDICTIONS>>>>")
    for k, n in product(K, norm):
        acc = evaluation(KNN(k, n, X_train, y_train, X_test), y_test)
        print(f"K = {k}, Norm = {n}, Accuracy = {acc}")
        df.loc[(df['K'] == k) & (df['norm'] == n), 'test-accuracy'] = acc

Uncomment the code below to run the main function (Remember to recomment the code before submitting).

In [ ]:
# Finally, call the main function
main()

In [ ]:
print(df.to_markdown())

Answer the following questions here:

1. How could having a larger dataset influence the performance of KNN?

- KNN actually does nothing in the training phase (it just collect the data), but when prediction, it must iterate through the whole dataset to find the nearest neighbors, which would increase the predicting time, aka. $O(pred) = O(n)$ w.r.t. $n$ is the size of the dataset.

2. Tabulate your results from `main()` in the table provided.

|    |   K | norm   |   val-accuracy |   test-accuracy |
|---:|----:|:-------|---------------:|----------------:|
|  0 |   3 | L1     |           0.94 |        0.884058 |
|  1 |   3 | L2     |           0.95 |        0.884058 |
|  2 |   3 | L-inf  |           0.94 |        0.898551 |
|  3 |   5 | L1     |           0.94 |        0.913043 |
|  4 |   5 | L2     |           0.93 |        0.898551 |
|  5 |   5 | L-inf  |           0.94 |        0.898551 |
|  6 |   7 | L1     |           0.93 |        0.898551 |
|  7 |   7 | L2     |           0.92 |        0.913043 |
|  8 |   7 | L-inf  |           0.93 |        0.898551 |

3. Finally, mention the best K and the norm combination you have settled upon and report the accuracy on the test set using that combination.

Setting `K` as 5 with `L1` metric, or setting `K` as 7 with `L2` metric, both get the highest score. However, the first combination may have a better (speed) performance.